In [4]:
import numpy as np
import pickle
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.sparse import csr_matrix
from configuration import *

## Identificação dos gêneros musicais

Carrega as amostras (espectro de cada trilha sonora) preparadas na etapa 1:

In [5]:
loader = np.load(TRACKS)
X = csr_matrix((loader['data'], loader['indices'], loader['indptr']), shape = loader['shape']).toarray()

In [7]:
n_samples, n_features = np.shape(X)
print("{} amostras x {} dimensões".format(n_samples, n_features))

5151 amostras x 1994 dimensões


Antes de prosseguir, precisamos checar a validade dos dados de entrada. Isto é, precisamos verificar se (1) todos os dados são numéricos e (2) se eles pertencem ao intervalo fechado $[0,1]$.

In [8]:
assert not np.all(np.isnan(X))
assert np.max(X) <= 1.0
assert np.min(X) >= 0.0

Utilizamos o algoritmo [_k-means_](https://en.wikipedia.org/wiki/K-means_clustering) para agrupar as trilhas de áudio similares entre si (cada uma representada como um vetor $N$ dimensional num espaço euclidiano, onde $N$ é a quantidade de rádios presentes nas amostras e o valor de cada componente é a frequência relativa com que cada trilha de áudio foi executada nessa rádio). Isso é feito concomitantemente a uma [análise de silhueta](http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html), que visa determinar a quantidade de gêneros (ie, _clusters_):

In [9]:
n_clusters_candidates = range(2,10) # A análise falha com apenas 1 cluster
chosen_ng = -1 # "ng" significa "quantidade (n) de gêneros"
chosen_ng_score = -1 # O valor mínimo do score no método da silhueta é -1
chosen_ng_labels = np.zeros(shape=(n_samples))

for n in n_clusters_candidates:
    
    # Executa k-means nas amostras
    model = KMeans(n_clusters=n, random_state=10).fit(X)

    # Determina o score médio (dentre os scores de cada amostra)
    score = silhouette_score(X, model.labels_)
    print("{0} clusters => score {1:.5f}".format(n, score))
    
    # Seleciona a quantidade de clusters (gêneros) que apresenta o maior score
    if (score > chosen_ng_score):
        chosen_ng = n
        chosen_ng_score = score
        chosen_ng_labels = model.labels_


if chosen_ng == -1:
    print("O algoritmo não pôde identificar a quantidade de gêneros existentes.")
else:
    print("# de gêneros escolhida: {}.".format(chosen_ng))

2 clusters => score 0.06085
3 clusters => score 0.02534
4 clusters => score 0.01248
5 clusters => score 0.02567
6 clusters => score -0.01399
7 clusters => score 0.01334
8 clusters => score -0.00724
9 clusters => score 0.03284
# de gêneros escolhida: 2.


## Persistência do mapeamento track_id $\to$ genre_id

Carregando o arquivo com os track_ids em ordem

In [10]:
track_ids = pickle.load(file(TRACK_IDS, 'rb'))

O dicionário abaixo associa cada track_id com o rótulo do gênero (_cluster_) do modelo selecionado na seção anterior:

In [11]:
mapper = dict(zip(track_ids, chosen_ng_labels))

Salva o mapeamento track_id:genre_id em disco, de modo que possa ser facilmente utilizado para classificar as trilhas de áudio e as rádios no _data-frame_ original.

In [12]:
pickle.dump(mapper, file(TRACK_GENRES_BINARY, 'wb'), pickle.HIGHEST_PROTOCOL)